In [ ]:
import meshtastic.serial_interface
from pubsub import pub
import json
import time

In [ ]:
interface = meshtastic.serial_interface.SerialInterface()
ourNode = interface.getNode('^local')

In [ ]:
from meshtastic.node import Node
from meshtastic.protobuf import config_pb2, admin_pb2

def getLoRaModemPreset(ourNode: Node):
    ourNode.requestConfig(admin_pb2.AdminMessage.LORA_CONFIG)
    ourNode.waitForConfig()
    modem_preset_val = ourNode.localConfig.lora.modem_preset
    return modem_preset_val

def setLoRaModemPreset(ourNode: Node, preset: int):
    ourNode.requestConfig(admin_pb2.AdminMessage.LORA_CONFIG)
    ourNode.waitForConfig()
    ourNode.localConfig.lora.modem_preset = preset
    ourNode.writeConfig("lora")

def toName(value: config_pb2.Config.LoRaConfig.ModemPreset.ValueType) -> str:
    return config_pb2.Config.LoRaConfig.ModemPreset.Name(value)

In [ ]:
def setOverrideDutyCycle(ourNode: Node, enabled: bool):
    ourNode.requestConfig(admin_pb2.AdminMessage.LORA_CONFIG)
    ourNode.waitForConfig()
    ourNode.localConfig.lora.override_duty_cycle = enabled
    ourNode.writeConfig("lora")

In [ ]:
def onReceive(packet, interface):
    try:
        if 'decoded' in packet and packet['decoded']['portnum'] == 'TEXT_MESSAGE_APP':
            message_bytes = packet['decoded']['payload']
            message_string = message_bytes.decode('utf-8')
            print(f"{message_string} \n> ", end="", flush=True)
    except KeyError as e:
        print(f"Error processing packet: {e}")

pub.subscribe(onReceive, 'meshtastic.receive')

def send_message(message):
    interface.sendText(message)

In [ ]:
setOverrideDutyCycle(ourNode, False)
time.sleep(1)

In [ ]:
for modem_preset in config_pb2.Config.LoRaConfig.ModemPreset.values():
    print(f"Setting LoRa Modem Preset to {toName(modem_preset)}")
    setLoRaModemPreset(ourNode, modem_preset)
    time.sleep(2)
    send_message(f"Modem {toName(modem_preset)}")
    time.sleep(1)

In [ ]:
interface.close()